# Wiele konstruktorów

Jeśli klasa posiada więcej niż jeden konstruktor to najczęściej każdy z nich prędzej czy później przypisuje wartości do tych samych atrybutów. Czasem te wartości są podawane przez parametry metody, czasem tworzone jako wartości domyślne a czasem są wynikiem bardziej skomplikowanych obliczeń. W efekcie dostajemy taką sytuację: 

In [ ]:
public Animal(IWorldMap map) {
   this.map = map;
   this.orientation = MapDirection.NORTH;
   this.position = new Vector2d(2 ,2);
}

public Animal(IWorldMap map, Vector2d position) {
    this.map = map;
    this.orientation = MapDirection.NORTH;
    this.position = position;
}

Widzimy, że same przypisania powtarzają się, co więcej niektóre są wręcz zduplikowane w obu konstruktorach. Żeby tego uniknąć i nie popełnić błędu np. zapominając o przypisaniu jednego z atrybutów możemy wywołać konstruktor z innego konstrutkora używając instrukcji `this()`.

In [ ]:
public Animal(IWorldMap map) {
   this(map, new Vector2d(2, 2));
}

Gdyby konstruktorów było jeszcze więcej, moglibyśmy kontynuować takie "zagnieżdżanie". Generalna zasada jest taka, że zawsze **mniejszy konstruktor wywołuje większy**.

In [ ]:
public Animal() {
    this(new RectangularMap(4, 4)); // tylko na potrzeby przykładu, w zadaniu raczej tak nie chcemy ;)
}

public Animal(IWorldMap map) {
   this(map, new Vector2d(2, 2));
}

public Animal(IWorldMap map, Vector2d position) {
    this.map = map;
    this.orientation = MapDirection.NORTH;
    this.position = position;
}

# Domyślne metody interfejsów

Czasem zdarza się, że metody, które realizują interfejs zależą tak naprawdę tylko od siebie nawzajem. Jeśli przyjrzymy się metodom w naszej klasie `RectangularMap`:

In [ ]:
public interface IWorldMap {

    boolean isOccupied(Vector2d position);
    
    Object objectAt(Vector2d position);
}

public class RectangularMap implements IWorldMap {

    private List<Animal> animals = new ArrayList<>();


    @Override
    public boolean isOccupied(Vector2d position) {
        return objectAt(position) != null;
    }


    @Override
    public Object objectAt(Vector2d position) {
        for (Animal animal : animals) {
            if (animal.isAt(position)) {
                return animal;
            }
        }
        return null;
    }
}

to zauważymy, że metoda `isOccupied()` nie korzysta w ogóle ze stanu mapy, tj. z żadnego z jej atrybutów. Nie korzysta również z żadnej z metod specyficznych tylko dla `RectangularMap`. Wywołuje jedynie inną metodę z interfejsu, którego sama jest częścią. W tego typu sytuacji możemy przenieść taką implementację do interfejsu - wystarczy tylko oznaczyć ją słowem `default` i Java pozwoli nam zdefiniować ciało metody, pomimo że jest to interfejs.

In [ ]:
public interface IWorldMap {

    default boolean isOccupied(Vector2d position) {
        return objectAt(position) != null;
    }
    
    Object objectAt(Vector2d position);
}


# Hermetyzacja a kolekcje

Wiemy już, że w przypadku atrybutów powinniśmy pamiętać by ustawiać ich modyfikatory na prywatne i ewentualnie dostarczać metod dostępu (getterów i/lub setterów). W przypadku kolekcji w teorii sprawa wygląda podobnie. Przyjrzymy się przykładowi:

In [ ]:
public class RectangularMap implements IWorldMap {

    private List<Animal> animals = new ArrayList<>();
    
    public List<Animal> getAnimals() {
        return animals;
    }
    
    public boolean place(Animal animal) {
        if (canMoveTo(animal.getPosition())) {
            animals.add(animal);
            return true;
        }
        return false;
  }
}

Wydaje się, że wszystko jest ok. Zabezpieczyliśmy naszą listę przed publicznym dostępem, nikt nam jej nie podmieni... Co jednak z jej zawartością? Z kodu wynika, że umieszczenie czegoś w liście może nastąpić tylko pod pewnymi warunkami (metoda `place()`), a tymczasem nikt nam nie zabroni zrobić tak:

In [ ]:
RectangularMap map = new RectangularMap();
map.getAnimals().add(new Animal(map, new Vector2d(666, 666)));

Jesli mapa miałaby np. rozmiar 10x10 to w ten sposób "zhakowalibyśmy" nasze własne rozwiązanie dodając zwierzę niemieszczące się na mapie. Jak sobie z tym poradzić? Najprostsze rozwiązanie to zwrócić kopię listy:

In [ ]:
public List<Animal> getAnimals() {
    return List.copyOf(animals); // uwaga: zwraca niemodyfikowalną listę, jesli chcemy modyfikować to new ArrayList<>(animals)
}

W ten sposób uchronimy się przed niechcianymi zmianami. Jeśli dodatkowo zależy nam na wydajności i nie chcemy za każdym razem kopiować listy to możemy zamiast tego zwrócić tzw. niemodyfikowalny widok, który jedynie opakowuje listę i przy próbie wywołania metod typu `add()` rzuca błędem.

In [ ]:
public List<Animal> getAnimals() {
    return Collections.unmodifiableList(animals);
}

# Strumienie dla kolekcji

Strumienie (Streams), które pokazywaliśmy już przy okazji omawiania poprzednich laborek można tworzyć na bazie różnych elementów. Najczęściej jednak będziemy z nich korzystać właśnie w połączeniu z kolekcjami. Każda z javowych kolekcji posiada metodę `stream()`, która przekształca kolekcję elementów typu `T` w `Stream<T>`. Możemy potem dowolnie dokładać do takiego strumienia operatory. Gdy w końcu dołożymy tzw. operator terminalny (np. `collect()` albo inny zwracający wartość a nie strumień), elementy kolekcji są kolejno przepuszczane przez wszystkie operatory napotkane po drodze. Sama źródłowa kolekcja nie jest przy tym w żaden sposób modyfikowana. 

W naszych zadaniach mieliśmy dwie czytelne sytuacje, gdzie strumienie mogły zostać użyte. Pierwsza z nich to metoda `isOccupied()`: 

In [ ]:
public boolean isOccupied(Vector2d position) {
    for (Animal animal : animals) {
        if (animal.isAt(position)) {
            return true;
        }
    }
    return false;
}

Całość metody sprowadza się do typowego "sprawdź czy którykolwiek element spełnia warunek". Korzystając ze streamów moglibymy tu zastosować operator `anyMatch()`, który dla każdego napotkanego elementu odpala predykat (funkcję zwracającą true/false) i zatrzymuje się gdy znajdzie pierwszy, dla którego predykat zwróci true.

In [ ]:
public boolean isOccupied(Vector2d position) {
    return animals.stream()
                  .anyMatch(animal -> animal.isAt(position));
}

Zwróćmy uwagę, że `anyMatch()` jest operatorem terminalnym, bo zwraca nie `Stream` tylko `boolean`. 

Oczywiście w naszym zadaniu można było również zaimplementować `isOccupied()` z wykorzystaniem `objectAt()` (jak w przykładach wyżej).

Nieco bardziej złożonym przypadkiem jest wspomniana metoda `objectAt()`. Jeśli chcielibyśmy napisać ją używając streamów, musimy zauważyć, że tu problem sprowadza się do "znajdź pierwszy element, który spełnia warunek". A więc wynikiem jest tu nie true/false tylko cały obiekt. Będziemy tu potrzebowali dwóch operatorów:
- `filter()` - przyjmujący predykat i przepuszczający dalej jedynie elementy, które go spełniają
- `findFirst()` - zatrzymujący strumień po pierwszym napotkanym elemencie (operacja terminalna)

In [ ]:
public Object objectAt(Vector2d position) {
    return animals.stream()
            .filter(animal -> animal.isAt(position))
            .findFirst()
            .orElse(null);
}

Po uruchomieniu strumień będzie wrzucał kolejne zwierzątka do filtra. Pierwsze z nich które przejdzie pomyślnie przez filtr (jego pozycja będzie zgodna) wpadnie do operatora `findFirst()`, który zakończy strumień i zwróci rezultat.

### Optional

Byłby to koniec przykładu, ale widzimy, że pojawia się tu jeszcze `orElse(null)`. Bierze się to z faktu, że `findFirst()` nie zwraca po prostu obiektu `Animal` tylko obiekt `Optional<Animal>`. Typ `Optional` wprowadzono w Javie 8 żeby usprawnić kontrolę nad sytuacjami gdy metoda może zwrócić `null`. Normalnie nic nas nie chroni przed nullami - jeśli nie spodziewamy się, że gdzieś pojawi się `null` i nie sprawdzimy tego to najprawdopodobniej prędzej czy później zobaczymy ulubiony komunikat programistów Javy, czyli `NullPointerException`. Stanie się tak, gdy spróbujemy wywołać na zmiennej wskazującej na `null` dowolną metodę.
Optionale mają za zadanie przede wszystkim uświadamiać nas, że dana metoda może nic nie zwrócić i należy taką sytuację obsłużyć. Mają też szereg metod podobnych do tych, które znamy ze Streamów (np. `map()`, `filter()`), co sprawia że często nawet nie musimy w ogóle takiego optionala rozpakowywać żeby przeprowadzić operacje na danych, które przechowuje. 

Ogólna zasada brzmi: **jeśli metoda może zwrócić null, powinna zamiast tego zwrócić `Optional`**. Jeśli będziemy trzymać się tej zasady, nasze programy staną się znacznie czytelniejsze i bardziej odporne na nasze błędy. 

Kierując się tą zasadą powinniśmy tak naprawdę zmienić nasz interfejs i przerobić metodę `objectAt()` tak by zwracała `Optional`:

In [ ]:
public Optional<? extends Object> objectAt(Vector2d position) {
    return animals.stream()
            .filter(animal -> animal.isAt(position))
            .findFirst();
}

Wtedy z kolei metoda `isOccupied()` powinna sprawdzić, czy zwrócony `Optional` zawiera coś w środku (nie sprawdzamy już nulla, bo nigdy nulla nie dostaniemy!):

In [ ]:
public boolean isOccupied(Vector2d position) {
    return objectAt(position).isPresent();
}

Inne przydatne metody na `Optionalu` to: `ifPresent(element -> ...)`, `orElse(wartośćJesliNull)` czy `get()` (pobiera wartość - powinna być używana tylko po sprawdzeniu, inaczej poleci NullPointerException). 

Jeśli z kolei chcemy opakować obiekt w `Optional` możemy użyć: `Optional.ofNullable(obiekt)` lub `Optional.of(obiekt)` w zalezności od tego, czy dopuszczamy by przekazany tam obiekt był nullem czy nie. 